## Fig 2: Dimensionless multiplicity function $M^2 n(M)/\bar\rho$ and linear halo bias $b(M)$

In [ ]:
# Standard imports
import sys
import numpy as np
import matplotlib.pyplot as plt

# Third-party imports
import camb # Calculates the linear matter power spectrum

# Halo Model library imports
sys.path.append('../src/')
import halomodel as halo
import cosmology

In [ ]:
# Set cosmological parameters
Omega_c = 0.25
Omega_b = 0.05
Omega_k = 0.0
h = 0.7
As = 1.97448e-9
ns = 0.96
w = -1.0
wa = 0.0
m_nu = 0.0 # in eV
sigma_8_set = True # If true uses the following value
sigma_8_in  = 0.7

In [ ]:
# CAMB parameters
zmax_CAMB = 2.
kmax_CAMB = 200.

# Redshifts
zs = [1., 0.] # CAMB reorders from high-z to low-z, so we define it like this from the start to avoid confusion

In [ ]:
# Sets cosmological parameters in camb to calculate the linear power spectrum
pars = camb.CAMBparams()
wb = Omega_b*h**2
wc = Omega_c*h**2

# This function sets standard and helium set using BBN consistency
pars.set_cosmology(ombh2=wb, omch2=wc, H0=100.*h, mnu=m_nu, omk=Omega_k)
pars.set_dark_energy(w=w, wa=wa, dark_energy_model='ppf') 
pars.InitPower.set_params(As=As, ns=ns, r=0)
pars.set_matter_power(redshifts=zs, kmax=kmax_CAMB) # Setup the linear matter power spectrum

# extract parameters from CAMB
Omega_m  = pars.omegam 
camb_results = camb.get_results(pars)
sigma_8 = (camb_results.get_sigma8()[zs.index(0.)]).item()

if sigma_8_set:
    scaling = (sigma_8_in/sigma_8)**2
    As *= scaling
    pars.InitPower.set_params(As=As, ns=ns, r=0)

Pk_lin = camb.get_matter_power_interpolator(pars, 
                                            nonlinear = False, 
                                            hubble_units = True, 
                                            k_hunit = True, 
                                            kmax = kmax_CAMB,
                                            var1 = camb.model.Transfer_tot,
                                            var2 = camb.model.Transfer_tot, 
                                            zmax = zmax_CAMB,
                                           )
Pk_lin = Pk_lin.P # Single out the linear P(k) interpolator
camb_results = camb.get_results(pars)
sigma_8 = (camb_results.get_sigma8()[zs.index(0.)]).item()

In [ ]:
# Initialise halo model
hmod = halo.model(z, Omega_m, name='Tinker et al. (2010)', Dv=330.)

In [ ]:
# Halo mass range [Msun/h]
Mmin = 1e9; Mmax = 1e15; nM = 129
Ms = np.logspace(np.log10(Mmin), np.log10(Mmax), nM)

# Lagrangian radii corresponding to halo masses; 
Rs = hmod.Lagrangian_radius(Ms)

In [ ]:
# Get sigma(R) from CAMB
sigmaRs_camb = cosmology.sigmaR(Rs, camb_results, integration_type='camb')
sigmaRs = sigmaRs_camb

In [ ]:
mass_functions = [
    'Sheth & Tormen (1999)',
    'Tinker et al. (2010)',
    'Despali et al. (2016)',
]

# Calculate b(M) and n(M)
# NOTE: Remove slow calculation with sigma(R) from P(k)
bs = []
Fs = []
for iz, z in enumerate(zs):
    for mass_function in mass_functions:
        print(mass_function, 'z = %1.1f'%(z))
        hmod = halo.model(z, Omega_m, name=mass_function, Dv=330.)
        b = hmod.linear_bias(Ms, sigmas=sigmaRs[iz])
        F = hmod.multiplicity_function(Ms, sigma=lambda R: camb_results.get_sigmaR(R, hubble_units=True, return_R_z=False)[iz])
        bs.append(b)
        Fs.append(F)

In [ ]:
# Make the plot
plt.subplots(2, 1, figsize=(5, 4), dpi=100, sharex=True)
n = len(mass_functions)

# Mass function
plt.subplot(2, 1, 1)
plt.plot(Ms[0], np.nan, color='black', ls='-', label=r'$z=0$')
plt.plot(Ms[0], np.nan, color='black', ls='--', label=r'$z=1$')
for i, mass_function in enumerate(mass_functions):
    plt.plot(Ms, Fs[i], ls='--', color='C%d'%i)
    plt.plot(Ms, Fs[i+n], ls='-', color='C%d'%i)
plt.xscale('log')
plt.gca().set_xticklabels([])
plt.ylim((0., 0.06))
plt.ylabel(r'$M^2 n(M)/\bar\rho$')
plt.xlim((Ms[0], Ms[-1]))
plt.legend()

# Linear bias
plt.subplot(2, 1, 2)
plt.axhline(1., color='black')
for i, mass_function in enumerate(mass_functions):
    plt.plot(Ms, bs[i], ls='--', color='C%d'%i)
    plt.plot(Ms, bs[i+n], ls='-', color='C%d'%i, label=mass_function)
plt.xlabel(r'$M / h^{-1} M_\odot$')
plt.xscale('log')
plt.ylim((0., 5.))
plt.ylabel(r'$b(M)$')

# Common x-axis
plt.xlim((Ms[0], Ms[-1]))
plt.legend()

plt.savefig('plots/hmf_and_bias.pdf',bbox_inches='tight')
plt.show()